# 再帰下降構文解析

In [7]:
def lex(source):
    source = source.replace('(', ' ( ')
    source = source.replace(')', ' ) ')
    source = source.split()
    return source

def parse_seq(tokens):
    if tokens[0]==')':
        return None
    else:
        p = parse(tokens)
        return Pair((p, parse_seq(tokens)))

def parse(tokens):
    token = tokens.pop(0)
    if token == '(':
        result = parse_seq(tokens)
        token = tokens.pop(0)
        if token == ')':
            return result
        raise SyntaxError('expected )')
    elif ')' == token:
        raise SyntaxError('unexpected )')
    else:
        return Pair(atom(token))
    
def atom(token):
    if token.isdigit():
        token = int(token)
    return token

def createPair(source):
    tokens = lex(source)
    pair = parse(tokens)
    return pair

# Class: Pair

In [183]:
import copy

class Pair:
    def __init__(self, data):
        types = [str, int]
        if type(data) in types or (type(data) == tuple and len(data) == 2):
            self.__data = data
        else:
            print('Illegal data type: data must be str, int, tuple')
            
    def getString(self):
        if type(self.__data) == str:
            return self.__data
        else:
            return 'Not a string data'
            
    def getNumber(self):
        if type(self.__data) == int:
            return self.__data
        else:
            return 'Not a number data'
            
    def getLeft(self):
        if type(self.__data) == tuple:
            return self.__data[0]
        else:
            return 'Not a leaf data'
            
    def getRight(self):
        if type(self.__data) == tuple:
            return self.__data[1]
        else:
            return 'Not a leaf data'
        
    def getData(self):
        return self.__data
    
    def atom(pair):
        types = [str, int]
        if pair is None:
            return Pair((None, None))
        else:
            data = pair.getData()
            if type(data) in types:
                return Pair((None, None))
        return None

    def eq(pair1, pair2):
        data1 = None if pair1 is None else pair1.getData()
        data2 = None if pair2 is None else pair2.getData()
        
        if data1 == data2:
            return Pair((None, None))
        else:
            return None
    
    def cond(pair1, pair2, pair3):
        if pair1 is not None:
            return pair2
        else:
            return pair3
        
    def eval(pair, env):
        if Pair.atom(pair) is not None:
            val = pair.getData()
            for e in env:
                if type(val)==str and val in e:
                    return e[val]
            return pair
        
        leftp = pair.getLeft() 
        if Pair.atom(leftp) is not None:
            left = leftp.getString()
            right = pair.getRight()

            if left == 'car':
                return Pair.eval(right.getLeft(), env).getLeft()
            elif left == 'cdr':
                return Pair.eval(right.getLeft(), env).getRight()
            elif left == 'cons':
                return Pair((Pair.eval(right.getLeft(), env), Pair.eval(right.getRight().getLeft(), env)))
            elif left == 'atom':
                return Pair.atom(Pair.eval(right.getLeft(), env))
            elif left == 'eq':
                return Pair.eq(Pair.eval(right.getLeft(), env), Pair.eval(right.getRight().getLeft(),env))
            elif left == 'if':
                return Pair.eval(Pair.cond(Pair.eval(right.getLeft(), env), right.getRight().getLeft(), right.getRight().getRight().getLeft()))
            elif left == '+':
                x = Pair.eval(right.getLeft(), env).getData()
                y = Pair.eval(right.getRight().getLeft(), env).getData()
                return Pair(x+y)
            elif left == 'square':
                x = Pair.eval(right.getLeft(), env).getData()
                return Pair(x**2)
            else:
                raise SyntaxError('cannot eval')
        else:
            leftleftp = leftp.getLeft()
            left = leftleftp.getString()
            if left == 'lambda':
                right = pair.getRight()
                args = Pair.pair2list2(right)
                return Pair.apply(leftp, args, env)
            
        
    def apply(l, args, env):
        largs = l.getRight().getLeft()
        larglist = Pair.pair2list(largs)
        body = l.getRight().getRight().getLeft()
        nenv = copy.deepcopy(env)
        e = {k: v for (k, v) in zip(larglist, args)}
        nenv.insert(0, e)
        return Pair.eval(body, nenv)
    
    def pair2list(pair):
        l = []
        while pair is not None:
            val = pair.getLeft().getData()
            l.append(val)
            pair = pair.getRight()
        return l
    
    def pair2list2(pair):
        l = []
        while pair is not None:
            val = pair.getLeft()
            l.append(val)
            pair = pair.getRight()
        return l

    def print(pair):
        if Pair.atom(pair) is not None:
            if pair is not None:
                if type(pair.getData()) == int:
                    print(pair.getData(), end='')
                if type(pair.getData()) == str:
                    print('\'' + pair.getData()+'\'', end='')
            else:
                print('nil', end='')
        else:
            print('(', end='')
            Pair.print(pair.getLeft())
            print(', ', end='')
            Pair.print(pair.getRight())
            print(')', end='')

# テスト

In [185]:
l = createPair('((lambda (x y) (cons (cons (car x) (car y)) z)) (1 2) (3 4))')
Pair.print(l)

c = createPair('(5 7)')
print()
Pair.print(c)
env = [{'z': c}]

print()
evaled = Pair.eval(l, env)
Pair.print(evaled)


(('lambda', (('x', ('y', nil)), (('cons', (('cons', (('car', ('x', nil)), (('car', ('y', nil)), nil))), ('z', nil))), nil))), ((1, (2, nil)), ((3, (4, nil)), nil)))
(5, (7, nil))
((1, 3), (5, (7, nil)))

In [157]:
l = createPair('(lambda (x y) (cons (cons (car x) (car y)) z))')
Pair.print(l)

a = createPair('(1 2)')
b = createPair('(3 4)')
print()
Pair.print(a)
print()
Pair.print(b)
args = [a, b]

c = createPair('(5 7)')
print()
Pair.print(c)
env = [{'z': c}]

print()
applied = Pair.apply(l, args, env)
Pair.print(applied)


('lambda', (('x', ('y', nil)), (('cons', (('cons', (('car', ('x', nil)), (('car', ('y', nil)), nil))), ('z', nil))), nil)))
(1, (2, nil))
(3, (4, nil))
(5, (7, nil))
((1, 3), (5, (7, nil)))

In [152]:
env = [{'x': Pair(100), 'y': Pair(200)}]

In [149]:
p = createPair('(1 2 3)')
Pair.print(p)
print()
Pair.pair2list(p)
# Pair.print(p)
# Pair.print(Pair.eval(p, env))

(1, (2, (3, nil)))


[1, 2, 3]

In [153]:
tokens = lex('(car (cons x y))')
p = parse(tokens)
Pair.print(p)

print()
Pair.print(Pair.eval(p, env))

('car', (('cons', ('x', ('y', nil))), nil))
100

In [6]:
tokens = lex('(+ x 2)')
p = parse(tokens)
Pair.print(p)

print()
Pair.print(Pair.eval(p, env))

('+', ('x', (2, nil)))
102

In [7]:
tokens = lex('(square (+ x y))')
p = parse(tokens)
Pair.print(p)

print()
Pair.print(Pair.eval(p, env))

('square', (('+', ('x', ('y', nil))), nil))
90000

# Parserがなかったころのテスト

In [338]:
env = {'x':100, 'y':200}

p1 = Pair(1)
p2 = Pair(2)
p3 = None
p4 = Pair((p1, p2))
p5 = Pair((p1, p3))
s1 = Pair('x')
s2 = Pair('y')
m_atom = Pair('atom')
m_car = Pair('car')
m_cdr = Pair('cdr')
m_cons = Pair('cons')
m_eq = Pair('eq')
m_if = Pair('if')
e1 = Pair((m_car, p4))
e2 = Pair((m_cdr, p4))
e3 = Pair((m_cons, p1))
e4 = Pair((m_atom, p1))
e5 = Pair((m_atom, p4))
r1 = Pair((s1, Pair((s2, None))))
r2 = Pair((s1, Pair((s1, None))))
e6 = Pair((m_cons, r1))
e7 = Pair((m_eq, r2))
i1 = Pair((p2, None))
i2 = Pair((p1, i1))
i3 = Pair((p3, i2))
i4 = Pair((p1, i2))
e8 = Pair((m_if, i3))
e9 = Pair((m_if, i4))

Pair.print(e1)
print("('car', (1, 2)):", Pair.eval(e1, env)) #1
print("('cdr', (1, 2)):", Pair.eval(e2, env)) #2
print("('atom', p1)", Pair.eval(e4, env) is not None) #True
print("('atom', p5)", Pair.eval(e5, env) is not None) #False
print("('cons', r1)", Pair.eval(e6, env)) #(x, y)
print("('eq', r2)", Pair.eval(e7, env)) #(x, x)
print("('if', e8)", Pair.eval(e8, env))
print("('if', e8)", Pair.eval(e9, env))

(car, (1, 2))('car', (1, 2)): None
('cdr', (1, 2)): None
('atom', p1) False
('atom', p5) False
('cons', r1) None
('eq', r2) None
('if', e8) None
('if', e8) None


In [204]:
p1 = Pair(1)
p2 = Pair(2)
p3 = None
p4 = Pair((p1, p2))
p5 = Pair((p1, p3))
# Pair.eq(p3, p3)
# Pair.atom(p4)
# Pair.cond(None, p1, p2).getData()
# Pair.cond(p1, p1, p2).getData()

In [184]:
p1 = Pair('1')
p2 = Pair(1)
p3 = Pair(1)
p4 = Pair((p1, p2))
p5 = Pair((p1, p3))

Pair.eq(p5, p4)

In [118]:
p1 = Pair('123')
p2 = Pair(123)
p3 = Pair((p1, p2))
p4 = Pair(None)

print()

print(p1.getString())
print(p2.getString())
print(p3.getString())
print(p4.getString())

print()

print(p1.getNumber())
print(p2.getNumber())
print(p3.getNumber())
print(p4.getNumber())

print()

print(p1.getLeft())
print(p2.getLeft())
print(p3.getLeft())
print(p4.getLeft())


123
Not a string data
None
Not a string data
None
Not a string data
None

Not a number data
None
123
Not a number data
None
Not a number data
None

Not a leaf data
None
Not a leaf data
None
Not a leaf data
None


# Reference

In [ ]:
def read_from(tokens):
    "トークンの列から式を読み込む。"
    if len(tokens) == 0:
        raise SyntaxError('unexpected EOF while reading')
    token = tokens.pop(0)
    if '(' == token:
        L = []
        while tokens[0] != ')':
            L.append(read_from(tokens))
        tokens.pop(0) # pop off ')'
        return L
    elif ')' == token:
        raise SyntaxError('unexpected )')
    else:
        return atom(token)
    
def atom(token):
    "数は数にし、それ以外のトークンはシンボルにする。"
    try: return int(token)
    except ValueError:
        try: return float(token)
        except ValueError:
            return Symbol(token)

# gomi

In [ ]:
def oldparse2(tokens):
    pair = None
    token = tokens.pop(0)
    if '(' == token:
        while tokens[0] != ')':
            pair = parse(tokens)
        tokens.pop(0)
#         return Pair()
    elif ')' == token:
        raise SyntaxError('unexpected )')
    else:
        return Pair(atom(token))



def oldparse(tokens):
    pair = None
    for c in reversed(tokens):
        if c == '(' or c==')':
            continue
            
        if c.isdigit():
            c = int(c)
        
        p = Pair(c)
        left = Pair((p, None))
        
        if pair is not None:
            pair = Pair((left.getLeft(), pair))
        else:
            pair = left
#         print(c)
    return pair

In [37]:
envt = {'x': 100}
nenv = envt.copy()
env2 = {'y': 200}
nenv.update(env2)
print(envt)
print(nenv)

{'x': 100}
{'x': 100, 'y': 200}


In [132]:
a = ['1','2','3']
b = [1,2,3]
dic = {k: v for (k, v) in zip(a, b)}
dic

{'1': 1, '2': 2, '3': 3}

In [180]:
l2 = createPair('((1 2) (3 4))')
Pair.print(l2)
print(Pair.pair2list2(l2))

l1 = createPair('(1 4)')
Pair.print(l1)
Pair.pair2list(l1)
print(Pair.pair2list(l1))

((1, (2, nil)), ((3, (4, nil)), nil))[<__main__.Pair object at 0x10c6a7748>, <__main__.Pair object at 0x10c6a7160>]
(1, (4, nil))[1, 4]
